In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
print("Pandas 버전:", pd.__version__)
print("NumPy 버전:", np.__version__)
print("re 버전:", re.__version__)

Pandas 버전: 2.2.2
NumPy 버전: 1.26.4
re 버전: 2.2.1


In [4]:
path = '/Users/seoeunseo/Desktop/DACON/제출파일/'           # 폴더 경로 지정

In [6]:
train_df = pd.read_csv(path+"train.csv")
test_df = pd.read_csv(path+"test.csv")
tcga2_df = pd.read_csv(path+"TCGA_2.csv").drop('Unnamed: 0', axis=1)

In [7]:
# 만약 유전체 컬럼만으로 데이터프레임이 구성되어 있고, 타겟 변수 'SUBCLASS'만 제외하면 된다면:
gene_columns = [col for col in train_df.columns if col != 'SUBCLASS']

# 또는, 유전체 컬럼의 이름이 미리 리스트로 제공된 경우:
# gene_columns = ['A2M', 'AAAS', 'ABCA1', ..., 'ZZZ3']  # 실제 유전체 컬럼 이름들로 구성된 리스트

def keep_first_variant_multiple_columns(df, columns_list):
    for column in columns_list:
        # 변이 코드가 문자열인지 확인하고, 그렇지 않으면 그대로 둡니다.
        df[column] = df[column].apply(lambda x: x.split(' ')[0] if isinstance(x, str) else x)
    return df

# 함수 적용
train_df = keep_first_variant_multiple_columns(train_df, gene_columns)
test_df = keep_first_variant_multiple_columns(test_df, gene_columns)

### TCGA_2 데이터 전처리

In [4]:
tcga2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590743 entries, 0 to 590742
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Hugo_Symbol             590743 non-null  object 
 1   Chromosome              590743 non-null  object 
 2   Variant_Classification  590743 non-null  object 
 3   HGVSp_Short             579689 non-null  object 
 4   Consequence             590743 non-null  object 
 5   TRANSCRIPT_STRAND       590743 non-null  float64
 6   SYMBOL_SOURCE           590743 non-null  object 
 7   BIOTYPE                 590743 non-null  object 
 8   CANONICAL               417458 non-null  object 
 9   APPRIS                  556567 non-null  object 
 10  SIFT                    363205 non-null  object 
 11  PolyPhen                363683 non-null  object 
 12  IMPACT                  590743 non-null  object 
 13  case_id                 590743 non-null  object 
 14  project_id          

In [8]:
tcga_df2 = tcga2_df.drop(['SYMBOL_SOURCE'], axis=1)

In [9]:
tcga_df2['SIFT'] =tcga_df2['SIFT'].fillna('tolerated_low_confidence(0.98)')

In [10]:
# 최빈값 (Mode) 구하기
mode_value = tcga2_df['APPRIS'].mode()[0]  # P1이 최빈값일 가능성이 높음

# 최빈값으로 결측값 채우기
tcga2_df['APPRIS'].fillna(mode_value, inplace=True)


/var/folders/s5/z4h29rk50jlbht219d69t3b00000gn/T/ipykernel_24289/2777643324.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tcga2_df['APPRIS'].fillna(mode_value, inplace=True)


In [11]:
tcga2_df['CANONICAL'] = tcga2_df['CANONICAL'].fillna('NO')

In [12]:
tcga2_df['PolyPhen'] = tcga2_df['PolyPhen'].fillna('unknown(0)')

In [13]:
# 'HGVSp_Short'를 제외한 나머지 열들의 결측값을 최빈값으로 채우기
columns_to_fill = tcga2_df.columns.difference(['HGVSp_Short'])  # 'HGVSp_Short'를 제외한 나머지 열 선택

for column in columns_to_fill:
    if tcga2_df[column].isnull().sum() > 0:  # 결측값이 있는 열만 처리
        mode_value = tcga2_df[column].mode()[0]  # 해당 열의 최빈값 구하기
        tcga2_df[column].fillna(mode_value, inplace=True)  # 최빈값으로 결측값 채우기

# 결과 확인
print(tcga2_df.isnull().sum())


/var/folders/s5/z4h29rk50jlbht219d69t3b00000gn/T/ipykernel_24289/3383463561.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tcga2_df[column].fillna(mode_value, inplace=True)  # 최빈값으로 결측값 채우기


Hugo_Symbol                   0
Chromosome                    0
Variant_Classification        0
HGVSp_Short               11054
Consequence                   0
TRANSCRIPT_STRAND             0
SYMBOL_SOURCE                 0
BIOTYPE                       0
CANONICAL                     0
APPRIS                        0
SIFT                          0
PolyPhen                      0
IMPACT                        0
case_id                       0
project_id                    0
dtype: int64


In [14]:
# 'splice'라는 문자열이 포함된 행 찾기
splice_rows = tcga2_df[tcga2_df['HGVSp_Short'].str.contains('splice', case=False, na=False)]

# 'splice'라는 문자열이 포함된 행을 제외한 나머지 데이터
tcga2_df = tcga2_df[~tcga2_df['HGVSp_Short'].str.contains('splice', case=False, na=False)]

# 결과 확인
tcga2_df.head(3)



,Hugo_Symbol,Chromosome,Variant_Classification,HGVSp_Short,Consequence,TRANSCRIPT_STRAND,SYMBOL_SOURCE,BIOTYPE,CANONICAL,APPRIS,SIFT,PolyPhen,IMPACT,case_id,project_id
0,WRAP73,chr1,Silent,p.P18=,synonymous_variant,-1.0,HGNC,protein_coding,YES,P1,deleterious(0),unknown(0),LOW,ce727e9e-325d-41d8-bfaf-0051418b9bef,TCGA-SKCM
1,TEKT2,chr1,Missense_Mutation,p.R177W,missense_variant,1.0,HGNC,protein_coding,YES,P1,deleterious(0.01),probably_damaging(0.977),MODERATE,ce727e9e-325d-41d8-bfaf-0051418b9bef,TCGA-SKCM
2,GJA5,chr1,Missense_Mutation,p.A111V,missense_variant,-1.0,HGNC,protein_coding,NO,P1,tolerated(0.16),benign(0.046),MODERATE,ce727e9e-325d-41d8-bfaf-0051418b9bef,TCGA-SKCM


In [15]:
# HGVSp_Short 열에서 NaN 값을 가진 행 제거
tcga2_df = tcga2_df.dropna(subset=['HGVSp_Short'])

# 결과 확인
tcga2_df.head(3)


,Hugo_Symbol,Chromosome,Variant_Classification,HGVSp_Short,Consequence,TRANSCRIPT_STRAND,SYMBOL_SOURCE,BIOTYPE,CANONICAL,APPRIS,SIFT,PolyPhen,IMPACT,case_id,project_id
0,WRAP73,chr1,Silent,p.P18=,synonymous_variant,-1.0,HGNC,protein_coding,YES,P1,deleterious(0),unknown(0),LOW,ce727e9e-325d-41d8-bfaf-0051418b9bef,TCGA-SKCM
1,TEKT2,chr1,Missense_Mutation,p.R177W,missense_variant,1.0,HGNC,protein_coding,YES,P1,deleterious(0.01),probably_damaging(0.977),MODERATE,ce727e9e-325d-41d8-bfaf-0051418b9bef,TCGA-SKCM
2,GJA5,chr1,Missense_Mutation,p.A111V,missense_variant,-1.0,HGNC,protein_coding,NO,P1,tolerated(0.16),benign(0.046),MODERATE,ce727e9e-325d-41d8-bfaf-0051418b9bef,TCGA-SKCM


In [16]:
# 'p.'을 없애고, '='를 앞에 있는 대문자 알파벳으로 바꾸는 함수
def modify_hgvsp(value):
    # 먼저 'p.' 제거
    value = value.replace('p.', '')
    return value

# HGVSp_Short 칼럼 수정
tcga2_df['HGVSp_Short'] = tcga2_df['HGVSp_Short'].apply(modify_hgvsp)

# 결과 확인
print(tcga2_df['HGVSp_Short'].head())


0      P18=
1     R177W
2     A111V
3      F76=
4    E1892K
Name: HGVSp_Short, dtype: object


In [17]:
# 'fs*숫자' 패턴에서 '*숫자' 부분을 제거하는 함수
def remove_fs_stop_code(value):
    # 'fs*숫자' 패턴에서 '*숫자' 부분을 제거
    return re.sub(r'fs\*\d+', 'fs', value)

# HGVSp_Short 칼럼에서 'fs*숫자' 제거
tcga2_df['HGVSp_Short'] = tcga2_df['HGVSp_Short'].apply(remove_fs_stop_code)

# 결과 확인
tcga2_df['HGVSp_Short'].head()


0      P18=
1     R177W
2     A111V
3      F76=
4    E1892K
Name: HGVSp_Short, dtype: object

In [18]:
# '='을 값의 맨 앞 알파벳으로 바꾸는 함수
def replace_equals_with_first_alpha(value):
    # 먼저 알파벳을 찾고 '='을 그 알파벳으로 바꿉니다
    first_alpha = re.search(r'[A-Za-z]', value)
    if first_alpha:
        value = value.replace('=', first_alpha.group(0))
    return value

# HGVSp_Short 칼럼 수정
tcga2_df['HGVSp_Short'] = tcga2_df['HGVSp_Short'].apply(replace_equals_with_first_alpha)

# 결과 확인
print(tcga2_df['HGVSp_Short'].head())


0      P18P
1     R177W
2     A111V
3      F76F
4    E1892K
Name: HGVSp_Short, dtype: object


In [19]:
# KIRP + KICH를 KIPAN으로 변환
tcga2_df['project_id'] = tcga2_df['project_id'].replace({'TCGA-KIRP': 'TCGA-KIPAN', 'TCGA-KICH': 'TCGA-KIPAN'})

# GBM을 GBMLGG로 변환
tcga2_df['project_id'] = tcga2_df['project_id'].replace({'TCGA-GBM': 'TCGA-GBMLGG'})

# ESCA + STAD를 STES로 변환
tcga2_df['project_id'] = tcga2_df['project_id'].replace({'TCGA-ESCA': 'TCGA-STES', 'TCGA-STAD': 'TCGA-STES'})

# 결과 확인
project_id_counts = tcga2_df['project_id'].value_counts()
print(project_id_counts)


project_id
TCGA-UCEC      147600
TCGA-SKCM       72846
TCGA-COAD       58434
TCGA-STES       49710
TCGA-LUAD       40973
TCGA-LUSC       36377
TCGA-BLCA       28257
TCGA-BRCA       21954
TCGA-HNSC       20435
TCGA-CESC       16322
TCGA-GBMLGG     13197
TCGA-LIHC       10435
TCGA-OV          9534
TCGA-LGG         8393
TCGA-KIRC        6272
TCGA-PRAD        5953
TCGA-PAAD        5797
TCGA-KIPAN       5330
TCGA-SARC        3961
TCGA-ACC         1929
TCGA-THCA        1646
TCGA-DLBC        1619
TCGA-LAML         958
TCGA-TGCT         585
TCGA-THYM         537
TCGA-PCPG         496
Name: count, dtype: int64


In [20]:
# 'TCGA-' 접두사 제거
tcga2_df['project_id'] = tcga2_df['project_id'].str.replace('TCGA-', '')

# 결과 확인
project_id_counts = tcga2_df['project_id'].value_counts()
print(project_id_counts)


project_id
UCEC      147600
SKCM       72846
COAD       58434
STES       49710
LUAD       40973
LUSC       36377
BLCA       28257
BRCA       21954
HNSC       20435
CESC       16322
GBMLGG     13197
LIHC       10435
OV          9534
LGG         8393
KIRC        6272
PRAD        5953
PAAD        5797
KIPAN       5330
SARC        3961
ACC         1929
THCA        1646
DLBC        1619
LAML         958
TGCT         585
THYM         537
PCPG         496
Name: count, dtype: int64


### 데이터 합치기 
### project_id만 사용

In [21]:
# Step 2: Melt train_df to have a long format with gene and mutation columns
train_melted = train_df.melt(id_vars=['ID'], var_name='Gene', value_name='Mutation')

# Step 3: Group tcga_df by 'case_id' and 'Hugo_Symbol', aggregate mutations
tcga_grouped = tcga2_df.groupby(['Hugo_Symbol', 'HGVSp_Short']).agg({
    'project_id': 'first',
}).reset_index()

# Step 4: Merge the melted train data with tcga_grouped based on gene and mutation
merged_train_df = pd.merge(train_melted, tcga_grouped, left_on=['Gene', 'Mutation'], right_on=['Hugo_Symbol', 'HGVSp_Short'], how='left')

# Step 5: Fill NaN values with 'WT'
merged_train_df = merged_train_df.fillna('WT')

# Step 6: Pivot the merged data back to wide format
final_train_df = merged_train_df.pivot(index='ID', columns='Gene', values=['Mutation','project_id'])

# Step 7: Flatten multi-level column names for easier access
final_train_df.columns = ['_'.join(col).strip() for col in final_train_df.columns.values]

# Step 8: Inspect the final train DataFrame
final_train_df = final_train_df.reset_index(drop=True)
# Step 9: SUBCLASS와 관련된 열 제거 (train_df에서만 존재)
final_train_df = final_train_df.drop(columns=[col for col in final_train_df.columns if 'SUBCLASS' in col])

# Step 10: Inspect the final train DataFrame after removing SUBCLASS-related columns
print(final_train_df.head())


  Mutation_A2M Mutation_AAAS Mutation_AADAT Mutation_AARS1 Mutation_ABAT  \
0           WT            WT             WT             WT            WT   
1           WT            WT             WT             WT            WT   
2        R895R            WT             WT             WT            WT   
3           WT            WT             WT             WT            WT   
4           WT            WT             WT             WT            WT   

  Mutation_ABCA1 Mutation_ABCA2 Mutation_ABCA3 Mutation_ABCA4 Mutation_ABCA5  \
0             WT             WT             WT             WT             WT   
1             WT             WT             WT             WT             WT   
2             WT             WT             WT             WT             WT   
3             WT             WT             WT             WT             WT   
4             WT             WT             WT             WT             WT   

   ... project_id_ZNF292 project_id_ZNF365 project_id_ZNF639  

In [22]:
import pandas as pd


# Step 2: Melt test_df to have a long format with gene and mutation columns
test_melted = test_df.melt(id_vars=['ID'], var_name='Gene', value_name='Mutation')

# Step 3: Group tcga_df by 'case_id' and 'Hugo_Symbol', aggregate mutations
tcga_grouped = tcga2_df.groupby(['Hugo_Symbol', 'HGVSp_Short']).agg({
    'project_id': 'first',
}).reset_index()

# Step 4: Merge the melted test data with tcga_grouped based on gene and mutation
merged_df = pd.merge(test_melted, tcga_grouped, left_on=['Gene', 'Mutation'], right_on=['Hugo_Symbol', 'HGVSp_Short'], how='left')
# Step 5: Fill NaN values with 'WT'
merged_df = merged_df.fillna('WT')
# Step 6: Pivot the merged data back to wide format
final_test_df = merged_df.pivot(index='ID', columns='Gene', values=['Mutation','project_id'])

# Step 7: Flatten multi-level column names for easier access
final_test_df.columns = ['_'.join(col).strip() for col in final_test_df.columns.values]

# Step 8: Save final result
final_test_df = final_test_df.reset_index(drop=True)

In [23]:
final_train_df['SUBCLASS'] = train_df['SUBCLASS']
final_train_df['ID'] = train_df['ID']
final_test_df['ID'] = test_df['ID']

In [24]:
final_train_df.head(3)

,Mutation_A2M,Mutation_AAAS,Mutation_AADAT,Mutation_AARS1,Mutation_ABAT,Mutation_ABCA1,Mutation_ABCA2,Mutation_ABCA3,Mutation_ABCA4,Mutation_ABCA5,...,project_id_ZNF639,project_id_ZNF707,project_id_ZNFX1,project_id_ZNRF4,project_id_ZPBP,project_id_ZW10,project_id_ZWINT,project_id_ZYX,SUBCLASS,ID
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,KIPAN,TRAIN_0000
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,SARC,TRAIN_0001
2,R895R,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,SKCM,TRAIN_0002


In [25]:
final_test_df.head(3)

,Mutation_A2M,Mutation_AAAS,Mutation_AADAT,Mutation_AARS1,Mutation_ABAT,Mutation_ABCA1,Mutation_ABCA2,Mutation_ABCA3,Mutation_ABCA4,Mutation_ABCA5,...,project_id_ZNF365,project_id_ZNF639,project_id_ZNF707,project_id_ZNFX1,project_id_ZNRF4,project_id_ZPBP,project_id_ZW10,project_id_ZWINT,project_id_ZYX,ID
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,TEST_0000
1,WT,WT,WT,WT,WT,R587Q,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,TEST_0001
2,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,TEST_0002


In [26]:
# train_df와 test_df의 컬럼을 비교하는 코드
train_columns = set(final_train_df.columns)
test_columns = set(final_test_df.columns)

# train_df에만 있고 test_df에는 없는 컬럼
missing_in_test = train_columns - test_columns

# test_df에만 있고 train_df에는 없는 컬럼
missing_in_train = test_columns - train_columns

print(f"train_df에만 있는 컬럼: {missing_in_test}")
print(f"test_df에만 있는 컬럼: {missing_in_train}")


train_df에만 있는 컬럼: {'SUBCLASS'}
test_df에만 있는 컬럼: set()


In [41]:
final_test_df.to_csv(path+'final_test5.csv')

In [42]:
final_train_df.to_csv(path+'final_train5.csv')